# Overview: checking metadata and samplesheet for all RUNS

## Preparation

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pathlib
import pysam
import pyfaidx
import warnings
import re
import argparse
import sys
import subprocess
from tqdm import tqdm 

tqdm.pandas()
warnings.filterwarnings("ignore")

main_outptudir = "/workdir/outdir/pipeline_output"
umt_distance = 0
analysis_output_version = "20250614"
name = "Vi"

path_to_main_output = os.path.join(main_outptudir, "data_analysis", name, analysis_output_version)
path_to_01_output = os.path.join(path_to_main_output, "01_output")
os.system(f"mkdir -p {path_to_01_output}")

rundf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name="Runs_tracking")
rundf.columns = ["Run", "SampleID", "UMI", "Panel_version", "Name", "amplicon_name", "serial"]

umi_rundf = rundf[(rundf["UMI"] == "Yes") & (rundf["Name"] == name) ]

collect_all_sample_sheets = [item for item in pathlib.Path("../experiments/").glob("SampleSheet_batch_*.csv")]

samplesheet = pd.DataFrame()
for file in collect_all_sample_sheets:
    df = pd.read_csv(file, sep=",")
    df["batch"] = file.name.replace(".csv", "")
    samplesheet = pd.concat([samplesheet, df], ignore_index=True)

samplesheet.columns = ["filename", "FASTQ1", "FASTQ2", "batch"]
samplesheet["Run"] = samplesheet["FASTQ1"].apply(lambda x: x.split("/")[5] if x.split("/")[5] != "2025" else x.split("/")[6])
samplesheet["SampleID"] = samplesheet["filename"].apply(lambda x: x.split("_")[0].split("-")[1])
print(f"All available runs: {", ".join(sorted(samplesheet['Run'].unique()))}")

All available runs: R7288, R7297, R7312, R7331, R7347, R7353, R7373, R7374, R7393, R7400


## Check if there is any missing FASTQ samples

In [2]:
missing_fastq = [item for item in umi_rundf.SampleID.unique() if item not in samplesheet.SampleID.unique()]

umi_rundf[umi_rundf.SampleID.isin(missing_fastq)]


,Run,SampleID,UMI,Panel_version,Name,amplicon_name,serial
754,R7434,TMPC4S1,Yes,CRC_panel version 1.2,Vi,"H1, H3",S1
755,R7434,TMPC4S2,Yes,CRC_panel version 1.2,Vi,"H1, H3",S2
756,R7434,TMPC4S3,Yes,CRC_panel version 1.2,Vi,"H1, H3",S3
758,R7434,TMPC4S5,Yes,CRC_panel version 1.2,Vi,"H1, H3",S5
759,R7434,TMPC4S6,Yes,CRC_panel version 1.2,Vi,"H1, H3",S6
760,R7434,TMPC4S7,Yes,CRC_panel version 1.2,Vi,"H1, H3",S7
761,R7434,TMPL7S1,Yes,Lung_panel version 1.1,Vi,"L1, L2, L3",S1
762,R7434,TMPL7S2,Yes,Lung_panel version 1.1,Vi,"L1, L2, L3",S2
763,R7434,TMPL7S3,Yes,Lung_panel version 1.1,Vi,"L1, L2, L3",S3
764,R7434,TMPL7S4,Yes,Lung_panel version 1.1,Vi,"L1, L2, L3",S4


## Match output paths to main metadata

since there is duplicated SampleID, we should merge labcode and Run together to make them unique

In [3]:
rerun = True
if (os.path.isfile(os.path.join(path_to_01_output, "UMI_runs_tracking.xlsx")) == False) | (rerun  == True):
    umi_rundf["uniqueID"] = umi_rundf[["SampleID", "Run"]].apply(lambda x: f"{x[0]}_{x[1]}", axis=1)
    samplesheet["uniqueID"] = samplesheet[["SampleID", "Run"]].apply(lambda x: f"{x[0]}_{x[1]}", axis=1)

    final_umi_rundf = umi_rundf.merge(samplesheet.drop(["SampleID", "Run"], axis = 1), left_on = "uniqueID", right_on = "uniqueID")
    final_umi_rundf["UMI_cov_file"] = final_umi_rundf[["batch", "filename"]].apply(lambda x: os.path.join(
        main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_COV", f"{x[1]}.connor_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov"
    ), axis = 1)
    final_umi_rundf["nonUMI_cov_file"] = final_umi_rundf[["batch", "filename"]].apply(lambda x: os.path.join(
        main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_COV", f"{x[1]}.CutAdapt_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov"
    ), axis = 1)

    final_umi_rundf["check_UMI"] = final_umi_rundf["UMI_cov_file"].apply(lambda x: os.path.exists(x))
    final_umi_rundf["check_nonUMI"] = final_umi_rundf["nonUMI_cov_file"].apply(lambda x: os.path.exists(x))

    final_umi_rundf["unmapped_bam"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT_UNMAPPED_BAM", f"{x[1]}_R1.UMIprocessed_bismark_bt2_pe.bam"), axis = 1
    )
    final_umi_rundf["connor_annotated"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "CONNOR_UMI_OUTPUT", f"{x[1]}.connor.fully_annotated.bam"), axis = 1
    )
    final_umi_rundf["connor_fastq"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "CONNOR_UMI_OUTPUT", f"{x[1]}.connor_R1.fastq.gz"), axis = 1
    )
    final_umi_rundf["umi_bam"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT", f"{x[1]}.connor_R1_bismark_bt2_pe.sorted.bam"), axis = 1
    )
    final_umi_rundf["non_umi_bam"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT", f"{x[1]}.CutAdapt_R1_bismark_bt2_pe.sorted.bam"), axis = 1
    )

    tqdm.pandas()
    for col in ["unmapped_bam", "connor_annotated", "connor_fastq", "umi_bam", "non_umi_bam" , "UMI_cov_file"]:
        print(f"working on column: {col}")
        def _get_line_count(x):
            check_file = os.path.exists(x)
            x_raw = x.replace(".sorted.bam", ".bam")
            if check_file == False:
                if os.path.exists(x_raw) == True: 
                    # print(f"sorted bam file is not available, but raw bam file is available, sorting and indexing it now...")
                    os.system(f"samtools sort -@ 15 {x_raw} -o {x}")
                    os.system(f"samtools index {x}")
                else:
                    # rint(f"File {x} does not exist, skipping line count for this file.")
                    return 0
            if ".bam" in x:
                result = subprocess.run(f"samtools view {x} -c", shell=True, capture_output=True, text=True)
            elif ".fastq.gz" in x:
                result = subprocess.run(f"zcat {x} | wc -l", shell=True, capture_output=True, text=True)
            else:
                result = subprocess.run(f"cat {x} | wc -l", shell=True, capture_output=True, text=True)
            return int(result.stdout.strip())
            
        final_umi_rundf[f"count_{col}"] = final_umi_rundf[col].progress_apply(_get_line_count)    

    final_umi_rundf.to_excel(os.path.join(path_to_01_output, "UMI_runs_tracking.xlsx"), index=False)
else:
    print("reading in saved processed UMI runs tracking file...")
    final_umi_rundf = pd.read_excel(os.path.join(path_to_01_output, "UMI_runs_tracking.xlsx"))

working on column: unmapped_bam


100%|██████████| 314/314 [02:38<00:00,  1.98it/s]


working on column: connor_annotated


100%|██████████| 314/314 [02:57<00:00,  1.77it/s]


working on column: connor_fastq


...

100%|██████████| 314/314 [00:00<00:00, 335.41it/s]


working on column: non_umi_bam


100%|██████████| 314/314 [02:23<00:00,  2.18it/s]


working on column: UMI_cov_file


100%|██████████| 314/314 [00:00<00:00, 478.60it/s]


# Helper functions

In [4]:
# panel_name = "Lung_panel version 1.1"
# cpgdf = pd.DataFrame()
# panel_name = panel_name.replace(" ", "_")
# paneldf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name=panel_name)
# paneldf[f"{genome_version}_org"] = paneldf[genome_version].values
# paneldf["lengthF"] = paneldf["Forward Primer Seq"].apply(lambda x: len(x))
# paneldf["lengthR"] = paneldf["Reverse Primer Seq"].apply(lambda x: len(x))

def _remove_primer_coords(x, lenF, lenR):
    chrom = x.split(":")[0]
    start = int(x.split(":")[1].split("-")[0])
    end = int(x.split(":")[1].split("-")[1])
    start = start + lenF - 1
    end = end - lenR + 1
    return f"{chrom}:{start}-{end}"

def get_refseq(path_to_all_fa, chrom, start, end):
        refseq = pyfaidx.Fasta(os.path.join(path_to_all_fa, "{}.fa".format(chrom)))
        return(str.upper(refseq.get_seq(name = "{}".format(chrom), start = start, end = end).seq))

path_to_all_fa = "/home/hieunguyen/resources/hg19"
genome_version = "hg19"

##### get list of all real cpg for this panel
all_cpgdf = dict()
all_cpgdf_with_primer = dict()
for panel_name in final_umi_rundf.Panel_version.unique():
    
    panel_name = panel_name.replace(" ", "_")
    paneldf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name=panel_name)
    paneldf[f"{genome_version}_org"] = paneldf[genome_version].values
    paneldf["lengthF"] = paneldf["Forward Primer Seq"].apply(lambda x: len(x))
    paneldf["lengthR"] = paneldf["Reverse Primer Seq"].apply(lambda x: len(x))
    paneldf[genome_version] = paneldf[[genome_version, "lengthF", "lengthR"]].apply(lambda x:_remove_primer_coords(x[0], x[1], x[2]), axis = 1)

    def generate_cpgdf(region_col, paneldf):
        outputdf = pd.DataFrame()
        print(f"working on panel: {panel_name}, with {len(paneldf[region_col].unique())} regions")
        for region in paneldf[region_col].unique():
            region_name = paneldf[paneldf[region_col] == region]["Amplicon"].values[0]
            chrom = region.split(":")[0].replace("Chr", "chr").replace("chr", "")
            start = int(region.split(":")[1].split("-")[0])
            end = int(region.split(":")[1].split("-")[1])
            refseq = pyfaidx.Fasta(os.path.join(path_to_all_fa, "chr{}.fa".format(chrom)))
            refseq_at_region = str.upper(refseq.get_seq(name = "chr{}".format(chrom), start = start, end = end).seq)

            all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_region)]
            cpg_coords = [f"chr{chrom}:{item + start}-{item + start + 1}" for item in all_cpg_in_cluster]

            tmp_outputdf = pd.DataFrame(data = cpg_coords, columns = ["CpG"])
            tmp_outputdf["region"] = region
            tmp_outputdf["region_name"] = region_name
            outputdf = pd.concat([outputdf, tmp_outputdf], axis = 0)
        print(f"outputdf shape after adding region {region}: {outputdf.shape}")
        return outputdf
    
    cpgdf = generate_cpgdf(genome_version, paneldf)
    cpgdf_with_primer = generate_cpgdf(f"{genome_version}_org", paneldf)
    
    cpgdf = cpgdf[["region", "CpG", "region_name"]]
    all_cpgdf[panel_name] = cpgdf.copy()

    cpgdf_with_primer = cpgdf_with_primer[["region", "CpG", "region_name"]]
    all_cpgdf_with_primer[panel_name] = cpgdf_with_primer.copy()


# for i in all_cpgdf.keys():
#     tmpdf1 = all_cpgdf[i].copy()
#     tmpdf2 = all_cpgdf_with_primer[i].copy()
#     assert tmpdf1.shape[0] == tmpdf2.shape[0], f"Error: {i} cpgdf and cpgdf_with_primer have different number of rows"


working on panel: Lung_panel_version_1.1, with 17 regions
outputdf shape after adding region chr5:40681300-40881381: (2429, 3)
working on panel: Lung_panel_version_1.1, with 17 regions
outputdf shape after adding region chr5:40681281-40881401: (2456, 3)
working on panel: CRC_panel_version_1.2, with 9 regions
outputdf shape after adding region chr17:75369589-75369671: (126, 3)
working on panel: CRC_panel_version_1.2, with 9 regions
outputdf shape after adding region chr17:75369563-75369688: (140, 3)
working on panel: CRC_panel_version_3.1, with 6 regions
outputdf shape after adding region chr16:85654146-85654238: (33, 3)
working on panel: CRC_panel_version_3.1, with 6 regions
outputdf shape after adding region chr16:85654124-85654259: (38, 3)
working on panel: CRC_panel_version_1.1, with 8 regions
outputdf shape after adding region chr17:75369629-75369739: (118, 3)
working on panel: CRC_panel_version_1.1, with 8 regions
outputdf shape after adding region chr17:75369604-75369763: (129, 3

# PROCESSING COV FILES

In [5]:

def generate_cov(input_cov_file, panel_name, sampleid, outputdir):
    os.system(f"mkdir -p {outputdir}")
    if os.path.exists(input_cov_file) == False:
        print(f"Input coverage file {input_cov_file} does not exist, skipping...")
        covdf = pd.DataFrame(columns=["chrom", "start", "end", "meth_density", "countC", "countT", "seq", "strand", "CpG", "check_context", "amplicon"])
        status = "File not available"
    else:
        print(f"Processing {input_cov_file} for panel {panel_name}...")
        # get list of CpG which are not in primer regions
        cpgdf = all_cpgdf[panel_name]
        covdf = pd.read_csv(input_cov_file, header = None, sep = "\t")
        covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
        if covdf.shape[0] != 0:
            covdf = covdf[covdf["chrom"].isin(["chrX", "chrY", "chrMT"]) == False]
            covdf = covdf[covdf["chrom"].str.contains("chrUn") == False]
            covdf = covdf[covdf["chrom"].str.contains("_") == False]
            covdf["seq"] = covdf[["chrom", "start"]].progress_apply(lambda x: get_refseq(path_to_all_fa= path_to_all_fa, 
                                                                    chrom = x[0], start = x[1], end = x[1] + 1), axis = 1)
            covdf["strand"] = covdf["seq"].apply(lambda x: "+" if x != "CG" else "-")
            covdf_raw = covdf.copy()
            covdf["start"] = covdf[["seq", "start"]].apply(lambda x: x[1] + 1 if x[0] != "CG" else x[1], axis = 1)

            covdf["chrom"] = covdf["chrom"].apply(lambda x: str(x))
            covdf["CpG"] = covdf[["chrom", "start"]].apply(lambda x: f"{str(x[0])}:{x[1]}-{x[1] + 1}", axis = 1)
            covdf["check_context"] = covdf["CpG"].apply(lambda x: "CpG_context" if x in cpgdf["CpG"].values else "False")
            covdf["amplicon"] = covdf["CpG"].apply(lambda x: cpgdf[cpgdf["CpG"] == x]["region_name"].values[0] if x in cpgdf["CpG"].values else "NA")
            status = "Processed"
        else:
            print(f"Coverage file {input_cov_file} is empty, skipping...")
            covdf = pd.DataFrame(columns=["chrom", "start", "end", "meth_density", "countC", "countT", "seq", "strand", "CpG", "check_context", "amplicon"])
            status = "Empty file"
        covdf.to_excel(os.path.join(outputdir, f"{sampleid}.xlsx"))
    return status, os.path.join(outputdir, f"{sampleid}.xlsx")

# mode = "UMI"  # or "ignore_UMI" 
for mode in ["UMI", "ignore_UMI"]:
    all_status = []
    all_finished_path = []
    for j in range(final_umi_rundf.shape[0]):
        panel_name = final_umi_rundf.Panel_version.values[j].replace(" ", "_")
        run = final_umi_rundf.Run.values[j]
        sampleid = final_umi_rundf.SampleID.values[j]
        if mode == "UMI":
            input_cov_file = final_umi_rundf.UMI_cov_file.values[j]
        elif mode == "ignore_UMI":
            input_cov_file = final_umi_rundf.nonUMI_cov_file.values[j]

        outputdir = os.path.join(path_to_01_output, "UMI_cov", run, panel_name, mode)
        status, output_cov_path = generate_cov(input_cov_file=input_cov_file, panel_name=panel_name, sampleid=sampleid, outputdir=outputdir)
        all_status.append(status)
        all_finished_path.append(output_cov_path)
    final_umi_rundf[f"{mode}_status"] = all_status
    final_umi_rundf[f"{mode}_processed_cov"] = all_finished_path

Processing /workdir/outdir/pipeline_output/SampleSheet_batch_20250604/UMT_DISTANCE_0/BISMARK_COV/1-TML1S1_S7501-S7701.connor_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov for panel Lung_panel_version_1.1...


100%|██████████| 4/4 [00:00<00:00, 5199.01it/s]


Processing /workdir/outdir/pipeline_output/SampleSheet_batch_20250604/UMT_DISTANCE_0/BISMARK_COV/2-TML1S2_S7502-S7702.connor_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov for panel Lung_panel_version_1.1...


100%|██████████| 4/4 [00:00<00:00, 3964.37it/s]


Processing /workdir/outdir/pipeline_output/SampleSheet_batch_20250604/UMT_DISTANCE_0/BISMARK_COV/3-TML1S3_S7503-S7703.connor_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov for panel Lung_panel_version_1.1...


...

100%|██████████| 32/32 [00:00<00:00, 11052.18it/s]


Processing /workdir/outdir/pipeline_output/SampleSheet_batch_20250612/UMT_DISTANCE_0/BISMARK_COV/13-TMPL6S6I3_S7593-S7793.CutAdapt_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov for panel Lung_panel_version_1.1...


100%|██████████| 45/45 [00:00<00:00, 11570.85it/s]


Processing /workdir/outdir/pipeline_output/SampleSheet_batch_20250612/UMT_DISTANCE_0/BISMARK_COV/14-TMPL6S7I3_S7594-S7794.CutAdapt_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov for panel Lung_panel_version_1.1...


100%|██████████| 45/45 [00:00<00:00, 11735.60it/s]


In [6]:
final_umi_rundf

,Run,SampleID,UMI,Panel_version,Name,amplicon_name,serial,uniqueID,filename,FASTQ1,...,count_unmapped_bam,count_connor_annotated,count_connor_fastq,count_umi_bam,count_non_umi_bam,count_UMI_cov_file,UMI_status,UMI_processed_cov,ignore_UMI_status,ignore_UMI_processed_cov
0,R7288,TML1S1,Yes,Lung_panel version 1.1,Vi,L1,S1,TML1S1_R7288,1-TML1S1_S7501-S7701,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/1-TML1S...,...,2645274,2639164,34584,340,2645274,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
1,R7288,TML1S2,Yes,Lung_panel version 1.1,Vi,L1,S2,TML1S2_R7288,2-TML1S2_S7502-S7702,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/2-TML1S...,...,2195246,2181957,32044,466,2195246,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
2,R7288,TML1S3,Yes,Lung_panel version 1.1,Vi,L1,S3,TML1S3_R7288,3-TML1S3_S7503-S7703,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/3-TML1S...,...,3528722,3519234,38204,542,3528722,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
3,R7288,TML1S4,Yes,Lung_panel version 1.1,Vi,L1,S4,TML1S4_R7288,4-TML1S4_S7504-S7704,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/4-TML1S...,...,3210054,3200836,36748,498,3210054,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
4,R7288,TML1S5,Yes,Lung_panel version 1.1,Vi,L1,S5,TML1S5_R7288,5-TML1S5_S7505-S7705,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/5-TML1S...,...,3048870,3039512,36012,596,3048870,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,R7400,TMPL6S3I3,Yes,Lung_panel version 1.1,Vi,"L3, L6, L12",S3,TMPL6S3I3_R7400,10-TMPL6S3I3_S7590-S7790,/mnt/GS-BACKUP05/FASTQ/2025/R7400/zatm/10-TMPL...,...,191490,189409,16324,262,191490,13,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
310,R7400,TMPL6S4I3,Yes,Lung_panel version 1.1,Vi,"L3, L6, L12",S4,TMPL6S4I3_R7400,11-TMPL6S4I3_S7591-S7791,/mnt/GS-BACKUP05/FASTQ/2025/R7400/zatm/11-TMPL...,...,363520,359986,17832,166,363520,35,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
311,R7400,TMPL6S5I3,Yes,Lung_panel version 1.1,Vi,"L3, L6, L12",S5,TMPL6S5I3_R7400,12-TMPL6S5I3_S7592-S7792,/mnt/GS-BACKUP05/FASTQ/2025/R7400/zatm/12-TMPL...,...,23070,22344,5640,16,23070,15,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
312,R7400,TMPL6S6I3,Yes,Lung_panel version 1.1,Vi,"L3, L6, L12",S6,TMPL6S6I3_R7400,13-TMPL6S6I3_S7593-S7793,/mnt/GS-BACKUP05/FASTQ/2025/R7400/zatm/13-TMPL...,...,1135294,1113771,33808,352,1135294,35,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...


# Main analysis

## Generate RUN cov file

In [7]:
run = "R7288"

all_inputs = final_umi_rundf[(final_umi_rundf["Run"] == run) & (final_umi_rundf["UMI_status"] == "Processed")]
all_inputs


,Run,SampleID,UMI,Panel_version,Name,amplicon_name,serial,uniqueID,filename,FASTQ1,...,count_unmapped_bam,count_connor_annotated,count_connor_fastq,count_umi_bam,count_non_umi_bam,count_UMI_cov_file,UMI_status,UMI_processed_cov,ignore_UMI_status,ignore_UMI_processed_cov
0,R7288,TML1S1,Yes,Lung_panel version 1.1,Vi,L1,S1,TML1S1_R7288,1-TML1S1_S7501-S7701,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/1-TML1S...,...,2645274,2639164,34584,340,2645274,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
1,R7288,TML1S2,Yes,Lung_panel version 1.1,Vi,L1,S2,TML1S2_R7288,2-TML1S2_S7502-S7702,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/2-TML1S...,...,2195246,2181957,32044,466,2195246,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
2,R7288,TML1S3,Yes,Lung_panel version 1.1,Vi,L1,S3,TML1S3_R7288,3-TML1S3_S7503-S7703,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/3-TML1S...,...,3528722,3519234,38204,542,3528722,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
3,R7288,TML1S4,Yes,Lung_panel version 1.1,Vi,L1,S4,TML1S4_R7288,4-TML1S4_S7504-S7704,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/4-TML1S...,...,3210054,3200836,36748,498,3210054,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
4,R7288,TML1S5,Yes,Lung_panel version 1.1,Vi,L1,S5,TML1S5_R7288,5-TML1S5_S7505-S7705,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/5-TML1S...,...,3048870,3039512,36012,596,3048870,4,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
5,R7288,TML2S1,Yes,Lung_panel version 1.1,Vi,L2,S1,TML2S1_R7288,6-TML2S1_S7506-S7706,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/6-TML2S...,...,2948432,2943330,23984,110,2948432,14,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
6,R7288,TML2S2,Yes,Lung_panel version 1.1,Vi,L2,S2,TML2S2_R7288,7-TML2S2_S7507-S7707,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/7-TML2S...,...,1807688,1804105,20268,80,1807688,15,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
7,R7288,TML2S3,Yes,Lung_panel version 1.1,Vi,L2,S3,TML2S3_R7288,8-TML2S3_S7508-S7708,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/8-TML2S...,...,2295430,2292264,21420,78,2295430,14,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
8,R7288,TML2S4,Yes,Lung_panel version 1.1,Vi,L2,S4,TML2S4_R7288,9-TML2S4_S7509-S7709,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/9-TML2S...,...,2364928,2360907,21884,60,2364928,14,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
9,R7288,TML2S5,Yes,Lung_panel version 1.1,Vi,L2,S5,TML2S5_R7288,10-TML2S5_S7510-S7710,/mnt/GS-BACKUP05/FASTQ/2025/R7288/zatm/10-TML2...,...,2859666,2856735,23616,124,2859666,14,Processed,/workdir/outdir/pipeline_output/data_analysis/...,Processed,/workdir/outdir/pipeline_output/data_analysis/...
